In [1]:
import os

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from sklearn.metrics import classification_report

In [2]:
working_dir = '/Users/rick/Desktop/Argonne_Summer_Work/WeatherNet/Weather_LSTM'

## Make train generator

In [3]:
train_df = pd.read_csv(working_dir+"/train_weather.csv")
train_df.index = train_df['Unnamed: 0'].values
train_df.drop(['Unnamed: 0'],axis=1,inplace=True)
print(train_df.shape)
train_df.head()

(2157, 310)


,Average 60 m wind direction (scalar) (units: deg),Average 60 m wind speed (scalar) (units: m/s),Standard deviation of 60 m wind direction (units: deg),Average 60 m temperature (units:deg C),Average 10 m wind direction (scalar) (units: deg),Average 10 m wind speed (scalar) (units: m/s),Standard deviation of 10 m wind direction (units: deg),Average 10 m temperature (units: deg C),Average dew point temperature (units: deg C),Average relative humidity (units: %),...,t9_2.0,t10_0.0,t10_1.0,t10_2.0,t11_0.0,t11_1.0,t11_2.0,t12_0.0,t12_1.0,t12_2.0
2020_01_01_12,0.205770,0.767677,0.233988,0.306061,0.002057,0.000060,0.500063,0.477164,0.070776,0.075093,...,0,1,0,0,1,0,0,1,0,0
2020_01_01_13,0.204268,0.797980,0.233134,0.339394,0.002054,0.000060,0.500068,0.502159,0.084475,0.075736,...,0,1,0,0,1,0,0,1,0,0
2020_01_01_14,0.190142,0.797980,0.238258,0.366667,0.001921,0.000060,0.500063,0.520336,0.099315,0.079722,...,0,1,0,0,1,0,0,1,0,0
2020_01_01_15,0.188539,0.828283,0.232280,0.375758,0.001905,0.000058,0.500061,0.522608,0.107306,0.082551,...,0,1,0,0,1,0,0,1,0,0
2020_01_01_16,0.185935,0.797980,0.237404,0.354545,0.001907,0.000053,0.500064,0.502159,0.116438,0.092966,...,0,1,0,0,1,0,0,1,0,0


In [4]:
train_df['p_target'].head(15)

2020_01_01_12    2
2020_01_01_13    2
2020_01_01_14    2
2020_01_01_15    1
2020_01_01_16    0
2020_01_01_17    0
2020_01_01_18    0
2020_01_01_19    0
2020_01_01_20    0
2020_01_01_21    0
2020_01_01_22    0
2020_01_01_23    2
2020_01_01_24    0
2020_01_02_00    0
2020_01_02_01    0
Name: p_target, dtype: int64

In [5]:
y_train = train_df["p_target"].values
y_train_hot = np.zeros((y_train.size, y_train.max()+1))
y_train_hot[np.arange(y_train.size),y_train] = 1

train_df.drop(['p_target'],axis=1,inplace=True)
X_train = train_df.values
print(X_train.shape)

(2157, 309)


In [6]:
time_steps = 1
train_gen = TimeseriesGenerator(X_train, y_train_hot,length=time_steps, batch_size=16)
print(len(train_gen))

135


In [7]:
# test generator to have corresponding labels
batch_0 = train_gen[0]
x, y = batch_0

print(x.shape)
print(y)

(16, 1, 309)
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


## Declare Model

In [8]:
from tensorflow.python.framework import ops
from tensorflow.python.keras import backend_config
from tensorflow.python.keras.optimizer_v2 import optimizer_v2
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import state_ops


class AdaMod(optimizer_v2.OptimizerV2):
    def __init__(self,
                 learning_rate=0.001,
                 beta_1=0.9,
                 beta_2=0.999,
                 beta_3=0.999,
                 epsilon=1e-8,
                 name='AdaMod',
                 **kwargs):
        super(AdaMod, self).__init__(name, **kwargs)
        self._set_hyper('learning_rate', kwargs.get('lr', learning_rate))
        self._set_hyper('decay', self._initial_decay)
        self._set_hyper('beta_1', beta_1)
        self._set_hyper('beta_2', beta_2)
        self._set_hyper('beta_3', beta_3)
        self.epsilon = epsilon or backend_config.epsilon()

    def _create_slots(self, var_list):
        # Create slots for the first and second moments.
        # Separate for-loops to respect the ordering of slot variables from v1.
        for var in var_list:
            self.add_slot(var, 'm')
        for var in var_list:
            self.add_slot(var, 'v')
        for var in var_list:
            self.add_slot(var, 'exp_avg_lr')

    def _prepare_local(self, var_device, var_dtype, apply_state):
        super(AdaMod, self)._prepare_local(var_device, var_dtype, apply_state)

        local_step = math_ops.cast(self.iterations + 1, var_dtype)
        beta_1_t = array_ops.identity(self._get_hyper('beta_1', var_dtype))
        beta_2_t = array_ops.identity(self._get_hyper('beta_2', var_dtype))
        beta_3_t = array_ops.identity(self._get_hyper('beta_3', var_dtype))
        beta_1_power = math_ops.pow(beta_1_t, local_step)
        beta_2_power = math_ops.pow(beta_2_t, local_step)
        lr = (apply_state[(var_device, var_dtype)]['lr_t'] *
              (math_ops.sqrt(1 - beta_2_power) / (1 - beta_1_power)))
        apply_state[(var_device, var_dtype)].update(dict(
            lr=lr,
            epsilon=ops.convert_to_tensor(self.epsilon, var_dtype),
            beta_1_t=beta_1_t,
            beta_1_power=beta_1_power,
            one_minus_beta_1_t=1 - beta_1_t,
            beta_2_t=beta_2_t,
            beta_2_power=beta_2_power,
            one_minus_beta_2_t=1 - beta_2_t,
            beta_3_t=beta_3_t,
        ))

    def set_weights(self, weights):
        params = self.weights
        num_vars = int((len(params) - 1) / 2)
        if len(weights) == 3 * num_vars + 1:
            weights = weights[:len(params)]
        super(AdaMod, self).set_weights(weights)

    def _resource_apply_dense(self, grad, var, apply_state=None):
        var_device, var_dtype = var.device, var.dtype.base_dtype
        coefficients = ((apply_state or {}).get((var_device, var_dtype))
                        or self._fallback_apply_state(var_device, var_dtype))

        # m_t = beta1 * m + (1 - beta1) * g_t
        m = self.get_slot(var, 'm')
        m_scaled_g_values = grad * coefficients['one_minus_beta_1_t']
        m_t = state_ops.assign(m, m * coefficients['beta_1_t'] + m_scaled_g_values,
                               use_locking=self._use_locking)

        # v_t = beta2 * v + (1 - beta2) * (g_t * g_t)
        v = self.get_slot(var, 'v')
        v_scaled_g_values = (grad * grad) * coefficients['one_minus_beta_2_t']
        v_t = state_ops.assign(v, v * coefficients['beta_2_t'] + v_scaled_g_values,
                               use_locking=self._use_locking)

        denom = math_ops.sqrt(v_t) + coefficients['epsilon']

        step_size = coefficients['lr'] / denom

        # exp_avg_lr.mul_(group['beta3']).add_(1 - group['beta3'], step_size)
        exp_avg_lr = self.get_slot(var, 'exp_avg_lr')
        exp_avg_lr = state_ops.assign(
            exp_avg_lr,
            exp_avg_lr * coefficients['beta_3_t'] + (1.0 - coefficients['beta_3_t']) * step_size,
            use_locking=self._use_locking)

        step_size = math_ops.minimum(step_size, exp_avg_lr)

        var_update = state_ops.assign_sub(
            var, m_t * step_size,
            use_locking=self._use_locking)

        return control_flow_ops.group(*[var_update, m_t, v_t, exp_avg_lr])

    def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
        raise RuntimeError('This optimizer does not support sparse gradients.')

    def get_config(self):
        config = super(AdaMod, self).get_config()
        config.update({
            'learning_rate': self._serialize_hyperparameter('learning_rate'),
            'decay': self._serialize_hyperparameter('decay'),
            'beta_1': self._serialize_hyperparameter('beta_1'),
            'beta_2': self._serialize_hyperparameter('beta_2'),
            'beta_3': self._serialize_hyperparameter('beta_3'),
            'epsilon': self.epsilon,
        })
        return config

In [9]:
i = Input(shape=(time_steps,X_train.shape[1]))

x = LSTM(32,kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-4),\
    bias_regularizer=regularizers.l2(1e-4),\
        recurrent_dropout=0.15,dropout=.15,return_sequences=False)(i)
x = Dropout(.2)(x)
y = Dense(3,activation="softmax",kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-4),\
    bias_regularizer=regularizers.l2(1e-4))(x)
    
model = Model(i, y) 
opt = Adam(.00001)
model.compile(loss='categorical_crossentropy',metrics=["acc"], optimizer=opt)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 309)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                43776     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 3)                 99        
Total params: 43,875
Trainable params: 43,875
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(train_gen, epochs=120,verbose=2)

Epoch 1/120
135/135 - 0s - loss: 1.3363 - acc: 0.4341
Epoch 2/120
135/135 - 0s - loss: 1.3229 - acc: 0.4726
Epoch 3/120
135/135 - 0s - loss: 1.3114 - acc: 0.4819
Epoch 4/120
135/135 - 0s - loss: 1.3011 - acc: 0.4930
Epoch 5/120
135/135 - 0s - loss: 1.2898 - acc: 0.4926
Epoch 6/120
135/135 - 0s - loss: 1.2793 - acc: 0.4963
Epoch 7/120
135/135 - 0s - loss: 1.2725 - acc: 0.4949
Epoch 8/120
135/135 - 0s - loss: 1.2629 - acc: 0.4940
Epoch 9/120
135/135 - 0s - loss: 1.2564 - acc: 0.4963
Epoch 10/120
135/135 - 0s - loss: 1.2431 - acc: 0.4944
Epoch 11/120
135/135 - 0s - loss: 1.2349 - acc: 0.4954
Epoch 12/120
135/135 - 0s - loss: 1.2286 - acc: 0.4944
Epoch 13/120
135/135 - 0s - loss: 1.2157 - acc: 0.4940
Epoch 14/120
135/135 - 0s - loss: 1.2127 - acc: 0.4991
Epoch 15/120
135/135 - 0s - loss: 1.2015 - acc: 0.4958
Epoch 16/120
135/135 - 0s - loss: 1.1943 - acc: 0.5028
Epoch 17/120
135/135 - 0s - loss: 1.1832 - acc: 0.5028
Epoch 18/120
135/135 - 0s - loss: 1.1736 - acc: 0.5097
Epoch 19/120
135/13

## Test Model

In [11]:
test_df = pd.read_csv(working_dir+"/test_weather.csv")
test_df.index = test_df['Unnamed: 0'].values
test_df.drop(['Unnamed: 0'],axis=1,inplace=True)

y_test = test_df["p_target"].values
y_test_hot = np.zeros((y_test.size, y_test.max()+1))
y_test_hot[np.arange(y_test.size),y_test] = 1

test_df.drop(['p_target'],axis=1,inplace=True)
X_test = test_df.values

test_gen = TimeseriesGenerator(X_test, y_test_hot,length=time_steps, batch_size=2)

In [12]:
yh = model.predict(test_gen)

target_names = ['low', 'mid', 'high']
print(classification_report(y_test[:yh.shape[0]], np.argmax(yh,axis=1), target_names=target_names))

              precision    recall  f1-score   support

         low       0.72      1.00      0.84        97
         mid       0.71      0.35      0.47        68
        high       0.83      0.80      0.81        84

    accuracy                           0.76       249
   macro avg       0.75      0.72      0.71       249
weighted avg       0.75      0.76      0.73       249

